In [6]:
import pennylane as qml
import pennylane as qml
from itertools import combinations
import torch

In [10]:
import pennylane as qml
from itertools import combinations


def real_amplitudes_layer(n_qubits, n_blocks):
    """Returns a real amplitudes circuit with some number of qubits and some number of blocks (i.e. depth)"""
    dev = qml.device("default.qubit", wires=n_qubits)

    @qml.qnode(dev, interface="torch", diff_method="adjoint")
    def real_amplitudes(inputs, weights):
        for i in range(n_qubits):
            qml.Hadamard(wires=i)
            qml.RY(inputs[i], wires=i)  # Angle encoding
        for i in range(n_blocks):
            for combo in combinations(list(range(0, n_qubits)), 2):
                qml.CNOT(wires=list(combo))
            for j in range(n_qubits):
                qml.RY(weights[i * n_qubits + j], wires=j)

        measured_qubits = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
        return measured_qubits

    weight_shapes = {"weights": n_blocks*n_qubits}
    layer = qml.qnn.TorchLayer(real_amplitudes, weight_shapes)
    return layer



In [11]:
custom_layer = real_amplitudes_layer(4, 4)

In [16]:
def real_amplitudes_layer(n_qubits, n_blocks):
    """Returns a real amplitudes circuit with some number of qubits and some number of blocks (i.e. depth)"""
    dev = qml.device("default.qubit", wires=n_qubits)

    @qml.qnode(dev, interface="torch", diff_method="adjoint")
    def real_amplitudes(inputs, weights):
        for i in range(n_qubits):
            qml.Hadamard(wires=i)
            qml.RY(inputs[i], wires=i)  # Angle encoding
        for i in range(n_blocks):
            for combo in combinations(list(range(0, n_qubits)), 2):
                qml.CNOT(wires=list(combo))
            for j in range(n_qubits):
                qml.RY(weights[i * n_qubits + j], wires=j)

        measured_qubits = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
        return measured_qubits

    weight_shapes = {"weights": n_blocks*n_qubits}
    layer = qml.qnn.TorchLayer(real_amplitudes, weight_shapes)
    return layer

In [18]:
import torch
import torchvision
import matplotlib.pyplot as plt
import pennylane as qml
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#import quantum_circuits

from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

training_data = torchvision.datasets.MNIST("root", train=True, download=True, transform=ToTensor())
testing_data = torchvision.datasets.MNIST("root", train=False, download=True, transform=ToTensor())

train_dataloader = DataLoader(training_data, batch_size = 500, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size = 100, shuffle=True)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.main = nn.Sequential(
            nn.Linear(in_features=784, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=6),
        )

        self.final = nn.Sequential(nn.Linear(in_features=6, out_features=10),
                                   nn.LogSoftmax(dim=1))

        #self.qlayer = quantum_circuits.real_amplitudes_layer(3, 1)
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.main(x)
        #x_1, x_2 = torch.split(x, 3, dim=1)
        #x_1 = self.qlayer(x_1)
        #x_2 = self.qlayer(x_2)
        #x = torch.cat([x_1, x_2], axis=1)
        x = self.final(x)
        return x

def train(num_epochs, network, optimizer, loss_function):
    train_accuracy = []
    train_loss = []
    for epoch in range(num_epochs):
        running_loss = 0
        running_accuracy = 0
        for idx, (inputs, labels) in enumerate(train_dataloader):
            optimizer.zero_grad()
            output = network(inputs)
            loss = loss_function(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_accuracy += (torch.max(output.data, 1)[1]==labels).sum()

        epoch_loss = running_loss / len(train_dataloader.dataset)
        epoch_acc = running_accuracy.float().item() / len(train_dataloader.dataset)
        print("Epoch: ", epoch, " Loss: ", epoch_loss, " Accuracy: ", epoch_acc)
        train_accuracy.append(epoch_acc)
        train_loss.append(epoch_loss)

network = Net()
num_epochs = 2
learning_rate = 0.01
optimizer = optim.Adam(network.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

train(num_epochs, network, optimizer, loss_function)

Epoch:  0  Loss:  0.0009653629439572493  Accuracy:  0.8500666666666666
Epoch:  1  Loss:  0.0002981178047756354  Accuracy:  0.9560833333333333
